# Data Science Project 2025/2026
#### Authors: Alexis Barros 2045719 & Vitor Remesso 2050519

## Problem Formulation
The dataset is a collection of flight delays and cancellations raging from the year 2019 to 2023, from the 'US Department of Transportation, Bureau of Transportation Statistics'. With this, we aim to predict arrival delay duration (in minutes) for a given flight based on scheduled times, airline, route, distance, and operational factors (using regression models). We will classify flights into three categories: 'On Time' (arrival delay < 15 minutes), 'Delayed' (arrival delay 15-30 minutes), and 'Long Delay' (arrival delay > 30 mins). In addition, we will identify patterns in operational performance by clustering, such as 'Airports based on delay behavior and traffic characteristics', and 'Airlines based on punctuality, delay causes, and route profiles'. Finally, we will test statistical hypothesis to determine if there are significant differences in delay patterns between different airlines, routes, or time periods.


Post-pandemic operations (2022-2023) suffered more carrier-related delays due to staff shortages than pre-pandemic (2019) operations.


In [31]:
# Libraries

import pandas as pd
import importlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
import numpy as np

# Data Analysis
## Data Load

In [32]:
import data_loader

importlib.reload(data_loader)

# Load the dataset
loader = data_loader.DataLoader()
df_flights = loader.data




Data Preview:
      FL_DATE                AIRLINE                AIRLINE_DOT AIRLINE_CODE  \
0  2019-01-09  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
1  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
2  2022-07-22  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
3  2023-03-06   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
4  2020-02-23       Spirit Air Lines       Spirit Air Lines: NK           NK   

   DOT_CODE  FL_NUMBER ORIGIN          ORIGIN_CITY DEST  \
0     19977       1562    FLL  Fort Lauderdale, FL  EWR   
1     19790       1149    MSP      Minneapolis, MN  SEA   
2     19977        459    DEN           Denver, CO  MSP   
3     19790       2295    MSP      Minneapolis, MN  SFO   
4     20416        407    MCO          Orlando, FL  DFW   

               DEST_CITY  CRS_DEP_TIME  DEP_TIME  DEP_DELAY  TAXI_OUT  \
0             Newark, NJ          1155    1151.0       -4.0      19.0   
1   

## Dataset Description

Each row in the dataset corresponds to a single scheduled commercial flight. The columns are:

### Flight-related:
- **FL_DATE**: Date of the flight (YYYY-MM-DD).
- **AIRLINE**: Airline name.
- **AIRLINE_DOT**: Airline name standardized by the U.S. Department of Transportation.
- **AIRLINE_CODE**: Two-letter IATA airline code.
- **DOT_CODE**: Unique DOT identifier for the airline.
- **FL_NUMBER**: Flight number assigned by the airline.
- **ORIGIN**: IATA airport code of the departure airport.
- **ORIGIN_CITY**: City and state of the departure airport.
- **DEST**: IATA airport code of the destination airport.
- **DEST_CITY**: City and state of the destination airport.

### Scheduled and actual times:
- **CRS_DEP_TIME**: Scheduled departure time.
- **DEP_TIME**: Actual departure time.
- **CRS_ARR_TIME**: Scheduled arrival time.
- **ARR_TIME**: Actual arrival time.

### Delay metrics:
- **DEP_DELAY**: Departure delay in minutes (actual departure minus scheduled departure). Negative values indicate early departure.
- **ARR_DELAY**: Arrival delay in minutes (actual arrival minus scheduled arrival). This variable is commonly used to define whether a flight is delayed.
- **TAXI_OUT**: Time (in minutes) from gate departure to wheels-off.
- **TAXI_IN**: Time (in minutes) from wheels-on to arrival at the gate.
- **WHEELS_OFF**: Time when the aircraft leaves the ground.
- **WHEELS_ON**: Time when the aircraft touches down.

### Flight duration and distance:
- **CRS_ELAPSED_TIME**: Scheduled total flight time (minutes).
- **ELAPSED_TIME**: Actual total flight time (minutes).
- **AIR_TIME**: Time spent in the air (minutes).
- **DISTANCE**: Distance between origin and destination airports (miles).

### Flight Status Indicators:
- **CANCELLED**: Binary indicator (1 if the flight was cancelled, 0 otherwise).
- **CANCELLATION_CODE**: Reason for cancellation (e.g., weather, carrier, airspace).
- **DIVERTED**: Binary indicator (1 if the flight was diverted to another airport).

### Causes of delay (in minutes):
These fields are typically populated when the arrival delay is 15 minutes or more:
- **DELAY_DUE_CARRIER**: Delay caused by airline-related issues.
- **DELAY_DUE_WEATHER**: Delay caused by weather conditions.
- **DELAY_DUE_NAS**: Delay due to the National Airspace System (e.g., congestion).
- **DELAY_DUE_SECURITY**: Delay caused by security issues.
- **DELAY_DUE_LATE_AIRCRAFT**: Delay due to the late arrival of the aircraft from a previous flight.


# Data Preprocessing / Cleaning
## Dropping irrelevant columns
We will drop columns that are not relevant for our analysis.

In [33]:
columns_to_drop = ['DEP_DELAY', 'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_SECURITY', 'DELAY_DUE_NAS','DELAY_DUE_LATE_AIRCRAFT', 'ARR_TIME', 'DEP_TIME', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_OUT', 'TAXI_IN', 'ELAPSED_TIME', 'AIR_TIME', 'CANCELLATION_CODE']
df_flights.drop(columns=columns_to_drop, inplace=True)
print(df_flights.head())

      FL_DATE                AIRLINE                AIRLINE_DOT AIRLINE_CODE  \
0  2019-01-09  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
1  2022-11-19   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
2  2022-07-22  United Air Lines Inc.  United Air Lines Inc.: UA           UA   
3  2023-03-06   Delta Air Lines Inc.   Delta Air Lines Inc.: DL           DL   
4  2020-02-23       Spirit Air Lines       Spirit Air Lines: NK           NK   

   DOT_CODE  FL_NUMBER ORIGIN          ORIGIN_CITY DEST  \
0     19977       1562    FLL  Fort Lauderdale, FL  EWR   
1     19790       1149    MSP      Minneapolis, MN  SEA   
2     19977        459    DEN           Denver, CO  MSP   
3     19790       2295    MSP      Minneapolis, MN  SFO   
4     20416        407    MCO          Orlando, FL  DFW   

               DEST_CITY  CRS_DEP_TIME  CRS_ARR_TIME  ARR_DELAY  CANCELLED  \
0             Newark, NJ          1155          1501      -14.0        0.0   
1       

Since we also are not interested in cancelled or diverted flights, we will drop those as well.

In [34]:
# First, let's check NA values BEFORE filtering
print("=" * 60)
print("BEFORE filtering cancelled/diverted flights:")
print("=" * 60)
print(f"Total flights: {len(df_flights)}")
print(f"\nNA values per column:")
print(df_flights.isnull().sum())
print(f"\nTotal rows with at least one NA value: {df_flights.isnull().any(axis=1).sum()}")
print(f"Percentage of rows with NA: {(df_flights.isnull().any(axis=1).sum() / len(df_flights) * 100):.2f}%")

# Check cancelled and diverted flights separately
print(f"\nCancelled flights: {df_flights['CANCELLED'].sum()}")
print(f"Diverted flights: {df_flights['DIVERTED'].sum()}")

# Check NA values in cancelled flights
cancelled_flights = df_flights[df_flights['CANCELLED'] == 1]
if len(cancelled_flights) > 0:
    print(f"\nNA values in CANCELLED flights:")
    print(cancelled_flights.isnull().sum())

# Check NA values in diverted flights
diverted_flights = df_flights[df_flights['DIVERTED'] == 1]
if len(diverted_flights) > 0:
    print(f"\nNA values in DIVERTED flights:")
    print(diverted_flights.isnull().sum())

print("\n" + "=" * 60)
print("NOW filtering out cancelled/diverted flights...")
print("=" * 60)

# Filter out cancelled and diverted flights (keep only those where both are 0)
df_flights = df_flights[(df_flights['CANCELLED'] == 0) & (df_flights['DIVERTED'] == 0)]

print(f"\nTotal flights after filtering: {len(df_flights)}")

# Drop the CANCELLED and DIVERTED columns as they are now all 0
df_flights = df_flights.drop(columns=['CANCELLED', 'DIVERTED'])


BEFORE filtering cancelled/diverted flights:
Total flights: 3000000

NA values per column:
FL_DATE                 0
AIRLINE                 0
AIRLINE_DOT             0
AIRLINE_CODE            0
DOT_CODE                0
FL_NUMBER               0
ORIGIN                  0
ORIGIN_CITY             0
DEST                    0
DEST_CITY               0
CRS_DEP_TIME            0
CRS_ARR_TIME            0
ARR_DELAY           86198
CANCELLED               0
DIVERTED                0
CRS_ELAPSED_TIME       14
DISTANCE                0
dtype: int64

Total rows with at least one NA value: 86198
Percentage of rows with NA: 2.87%

Cancelled flights: 79140.0
Diverted flights: 7056.0

NA values in CANCELLED flights:
FL_DATE                 0
AIRLINE                 0
AIRLINE_DOT             0
AIRLINE_CODE            0
DOT_CODE                0
FL_NUMBER               0
ORIGIN                  0
ORIGIN_CITY             0
DEST                    0
DEST_CITY               0
CRS_DEP_TIME            0
CR

## Handling Missing Values
We will check how many missing values exist in each column.

**Important Note**: Missing values in flight data typically occur when:
- A flight is **cancelled** (no actual times, no delays, no air time)
- A flight is **diverted** (arrival information is different)
- Data recording errors

Since we've already filtered out cancelled and diverted flights, we expect most NA values to be gone.

In [35]:
# Check missing values before dropping
print("Missing values per column:")
print(df_flights.isnull().sum())
print(f"\nTotal rows with at least one NA value: {df_flights.isnull().any(axis=1).sum()}")
print(f"Percentage of rows with NA: {(df_flights.isnull().any(axis=1).sum() / len(df_flights) * 100):.2f}%")

# If there are any NA values, let's see which rows have them
if df_flights.isnull().any(axis=1).sum() > 0:
    print("\nRows with NA values:")
    print(df_flights[df_flights.isnull().any(axis=1)].head(10))
    df_flights.dropna(inplace=True)
    print(f"\nRows remaining after dropping NA values: {len(df_flights)}")
else:
    print("\n✓ No missing values found! All flights that were not cancelled or diverted have complete data.")


Missing values per column:
FL_DATE             0
AIRLINE             0
AIRLINE_DOT         0
AIRLINE_CODE        0
DOT_CODE            0
FL_NUMBER           0
ORIGIN              0
ORIGIN_CITY         0
DEST                0
DEST_CITY           0
CRS_DEP_TIME        0
CRS_ARR_TIME        0
ARR_DELAY           2
CRS_ELAPSED_TIME    0
DISTANCE            0
dtype: int64

Total rows with at least one NA value: 2
Percentage of rows with NA: 0.00%

Rows with NA values:
            FL_DATE                 AIRLINE                 AIRLINE_DOT  \
918330   2023-05-21        Republic Airline        Republic Airline: YX   
1302862  2022-12-21  Frontier Airlines Inc.  Frontier Airlines Inc.: F9   

        AIRLINE_CODE  DOT_CODE  FL_NUMBER ORIGIN   ORIGIN_CITY DEST  \
918330            YX     20452       5612    LGA  New York, NY  BNA   
1302862           F9     20436        777    DEN    Denver, CO  LAS   

             DEST_CITY  CRS_DEP_TIME  CRS_ARR_TIME  ARR_DELAY  \
918330   Nashville, TN     